Installing all the packages required for the analysis

In [3]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install msgpack

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


Importing the required modules

In [62]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from datetime import date, timedelta
import pandas as pd

Creating the link between chrome, cromedriver and selenium

In [63]:
options = Options()
options.binary_location = "C:\Application Data\Google\Chrome\Application\chrome.exe"
driver = webdriver.Chrome(chrome_options = options, executable_path=r'C:\chromedriver\chromedriver.exe')
driver.get('http://google.com/')
print("Chrome Browser Invoked")

<ipython-input-63-55324ea3f24f>:3: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options = options, executable_path=r'C:\chromedriver\chromedriver.exe')


Chrome Browser Invoked


Directing the chromedriver towards the webpage of interest: FlightAware - Cancelled flights

In [64]:
url = 'https://uk.flightaware.com/live/cancelled/'
driver.get(url)
#setting a sleeping time to add delay in the execution of the program
time.sleep(3)

Clicking accept on the cookies' widgets

In [65]:
driver.find_element_by_css_selector('button.sc-ifAKCX.ljEJIv').click()
time.sleep(3)

driver.find_element_by_css_selector('span#cookieDisclaimerButtonText').click()
time.sleep(3)

Creating a list of all the airlines present in the main page of FlightAware

In [66]:
#locating the table
tables1 = driver.find_elements_by_id("airline-cancel-table")

for table in tables1:
    print('ok')

#finding the right classes in the html script
all_rows1 = table.find_elements_by_class_name("smallrow1")
all_rows2 = table.find_elements_by_class_name("smallrow2")

all_rows = all_rows1+all_rows2

airlines = []
for table in tables1:
    for row in all_rows:
        for air in row.find_elements_by_tag_name('a'):
            airlines.append(air.text)

ok


In [70]:
len(airlines)

100

In [ ]:
print(airlines)

Defining the start column of the historical cancelled flights dataset

In [68]:
def initiate_file(date_today):
    
    date_yesterday = date_today - timedelta(days=1) #more accuracy, all the flights of the day accouted for 
    
    url_yesterday = 'https://uk.flightaware.com/live/cancelled/yesterday'
    driver.get(url_yesterday)
    time.sleep(3)
    
    f = open('Flights.csv', 'w', encoding = 'utf-8')
    f.write(f'{date_yesterday}\nAirline_origin,\tAirport_origin_for_each_airline\n')

    for air in airlines:
        driver.find_element_by_link_text(air)
        origin_airline = driver.find_element_by_link_text(air).text
        driver.find_element_by_link_text(air).click()
        
        for start_plane in driver.find_elements_by_class_name("hint")[::2]:
                f.write(f"{origin_airline},{start_plane.text}\n") #either tab or teh comma
        
        try: #since some airlines have multiple pages of flight cancellations
            while True:
                driver.find_element_by_link_text('Next 20').click()

                for start_plane in driver.find_elements_by_class_name("hint")[::2]: #considering every other element because we are only considering the origin
                    f.write(f"{origin_airline},{start_plane.text}\n")
                time.sleep(3)

        except:
            driver.get(url)
            time.sleep(3)     
       
        
        time.sleep(3)
    f.close()   

1. look for the yesterday's data
2. change date
3. delete the empty cells (R or Python)

Function to be called only one time

In [69]:
initiate_file(date.today())

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"Thai Airways"}
  (Session info: chrome=94.0.4606.61)
  (Driver info: chromedriver=2.41.578737 (49da6702b16031c40d63e5618de03a32ff6c197e),platform=Windows NT 10.0.19042 x86_64)


Defining the function to append the data from each airline, everyday

In [35]:
def get_data(date_today):
    
    date_yesterday = date_today - timedelta(days=1) #more accuracy, all the flights of the day accouted for 
    
    url_yesterday = 'https://uk.flightaware.com/live/cancelled/yesterday'
    driver.get(url_yesterday)
    
    f = open('Flights.csv', 'a', encoding = 'utf-8')

    f.write(f'{date_yesterday}\nAirline_origin,\tAirport_origin_for_each_airline\n')

    for air in airlines:
        driver.find_element_by_link_text(air)
        origin_airline = driver.find_element_by_link_text(air).text
        driver.find_element_by_link_text(air).click()
        
        for start_plane in driver.find_elements_by_class_name("hint")[::2]:
                f.write(f"{origin_airline},{start_plane.text}\n") #either tab or teh comma
        
        try: #since some airlines have multiple pages of flight cancellations
            while True:
                driver.find_element_by_link_text('Next 20').click()

                for start_plane in driver.find_elements_by_class_name("hint")[::2]: #considering every other element because we are only considering the origin
                    f.write(f"{origin_airline},{start_plane.text}\n")
                time.sleep(3)

        except:
            driver.get(url)
            time.sleep(3)     
       
        
        time.sleep(3)
    f.close()   

Function to be called everyday at anytime  e.g. 12:00 a.m. because the data are from yesterday

In [ ]:
get_data(date.today())

Preview of the content of the cvs file

In [33]:
df = pd.read_csv("Flights.csv", sep=";") # work on the separator
# shows top 10 rows
df.head(10)

,2021-10-13
0,Airline_origin\tAirport_origin_for_each_airline
1,"China Eastern,\tQingdao Jiaodong Int'l"
2,"China Eastern,\tXishuangbanna Gasa"
3,"China Eastern,\tChengdu Shuangliu Int'l"
4,"China Eastern,\tShenzhen Bao'an Int'l"
5,"China Eastern,\tChangzhou Benniu"
6,"China Eastern,\tHarbin Taiping Int'l"
7,"China Eastern,\tYinchuan Helanshan"
8,"China Eastern,\tShenzhen Bao'an Int'l"
9,"China Eastern,\tXi'an Xianyang Int'l"


In [1]:
import os, sys
print(os.path.dirname(sys.executable))

C:\Users\fedef\anaconda3
